In [2]:
# import xlrd
import re
from datetime import datetime as dt
import os
import pandas as pd
import numpy as np

"""
Reads from the ecg file and the excel file, and creates two new files. One containing the raw ECG signals, and the
other containing the R-peaks, each line corresponding to the same line in the other file. 1 for R-peak and 0 otherwise.
"""

total_count = 10000000  # Maximum lines to copy over
count = 0  # Current number of lines

xl_file = os.path.join('..', 'Signal', 'RAT #01_2021_baseline_EPOCH DATA.xlsx')  # Data files
ecg_file_name = os.path.join('..', 'ECG_Data', 'RAT #01_2021_baseline.ascii')
out_file = os.path.join('..', 'Training', 'rat_train.txt')
'''
Rat Val: 'RAT #12_2016_WK4.ascii'
Rat Train: RAT #01_2021_baseline.ascii

Mouse Train: T22 - 2 hour data.ascii
Mouse Val: T21_transition example3_900s.ascii
'''

"\nRat Val: 'RAT #12_2016_WK4.ascii'\nRat Train: RAT #01_2021_baseline.ascii\n\nMouse Train: T22 - 2 hour data.ascii\nMouse Val: T21_transition example3_900s.ascii\n"

In [3]:
df_excel = pd.read_excel(xl_file, sheet_name=6, usecols=['Date'])

C:\Users\tonyz\anaconda3\envs\HeartRateVariability\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tonyz\anaconda3\envs\HeartRateVariability\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
df = pd.read_csv(ecg_file_name, comment='#')
df.columns = df.columns.str.replace(' ', '') 

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df['Time'] = pd.to_datetime(df['Time'], format="%m/%d/%Y %I:%M:%S.%f %p", exact=False)
# date2 = df_excel['Date']
# date1, date2

In [ ]:
df = df.merge(df_excel, left_on='Time', right_on='Date', how='outer')

In [ ]:
keys = list(df.columns.values)
print(keys)

In [ ]:
markings = 1 - df[keys[2]].isna().to_numpy()
markings[1:][markings[:-1]==markings[1:]] = 0
markings

In [ ]:
df = df[[keys[1], keys[2]]]

In [ ]:
markings = 1 - df[keys[2]].isna().to_numpy()
markings[1:][markings[:-1]==markings[1:]] = 0
df[keys[2]] = markings
df[keys[1]] = pd.to_numeric(df[keys[1]], errors ='coerce').fillna(0)

In [11]:
df.to_csv(out_file, sep=' ', header=False, index=False)